In [4]:
# Cell 1: Imports and Setup
import copy
import random
import numpy as np
import os
import glob
import io
import base64
from rich import print
from IPython.display import HTML, display, Video

# Gym and Highway-env
import gymnasium as gym
from gymnasium.wrappers import RecordVideo

# DiLu Imports (Ensure these are in your path)
from dilu.scenario.envScenario import EnvScenario
from dilu.driver_agent.driverAgent import DriverAgent
from dilu.driver_agent.vectorStore import DrivingMemory
from dilu.driver_agent.reflectionAgent import ReflectionAgent

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# --- Configuration Dictionary (Replacing config.yaml) ---
config = {
    # API Configuration (Fill in your actual keys)
    #'OPENAI_API_TYPE': 'openai',  # or 'azure'
    #'OPENAI_KEY': 'sk-proj-YOUR_OPENAI_KEY_HERE',
    #'OPENAI_CHAT_MODEL': 'gpt-4-turbo',
    'OPENAI_API_TYPE': 'ollama',  # or 'ollama' for local small models
    'OPENAI_KEY': 'ollama',
    'OPENAI_CHAT_MODEL': 'qwen2.5:14b', # use a model pulled by ollama (e.g. 'qwen2.5:14b' or 'llama3.1')

    # Azure specific (leave empty if using openai)
    'AZURE_API_VERSION': '',
    'AZURE_API_BASE': '',
    'AZURE_API_KEY': '',
    'AZURE_CHAT_DEPLOY_NAME': '',
    'AZURE_EMBED_DEPLOY_NAME': '',

    # Simulation Settings
    'vehicle_count': 50,
    'other_vehicle_type': 'highway_env.vehicle.behavior.IDMVehicle',
    'simulation_duration': 40, # Steps per episode
    'vehicles_density': 2,
    'episodes_num': 1, # Number of episodes to run in this cell

    # DiLu Specific
    'reflection_module': True,
    'memory_path': './memory_db', # Local path for vector store
    'few_shot_num': 3,
    'result_folder': './results'
}

# --- Environment Setup Function ---
def setup_env_vars(config):
    if config['OPENAI_API_TYPE'] == 'azure':
        os.environ["OPENAI_API_TYPE"] = config['OPENAI_API_TYPE']
        os.environ["OPENAI_API_VERSION"] = config['AZURE_API_VERSION']
        os.environ["OPENAI_API_BASE"] = config['AZURE_API_BASE']
        os.environ["OPENAI_API_KEY"] = config['AZURE_API_KEY']
        os.environ["AZURE_CHAT_DEPLOY_NAME"] = config['AZURE_CHAT_DEPLOY_NAME']
        os.environ["AZURE_EMBED_DEPLOY_NAME"] = config['AZURE_EMBED_DEPLOY_NAME']
    elif config['OPENAI_API_TYPE'] == 'openai':
        os.environ["OPENAI_API_TYPE"] = config['OPENAI_API_TYPE']
        os.environ["OPENAI_API_KEY"] = config['OPENAI_KEY']
        os.environ["OPENAI_CHAT_MODEL"] = config['OPENAI_CHAT_MODEL']

    # Case 3: Ollama (NEW)
    elif config['OPENAI_API_TYPE'] == 'ollama':
        # We tell the system it's "openai" so libraries behave normally
        os.environ["OPENAI_API_TYPE"] = 'openai'

        # Point to local host
        os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"  # Used by LangChain/Old libs
        os.environ["OPENAI_BASE_URL"] = "http://localhost:11434/v1"  # Used by Newer OpenAI libs

        # Dummy key and model
        os.environ["OPENAI_API_KEY"] = "ollama"
        os.environ["OPENAI_CHAT_MODEL"] = config['OPENAI_CHAT_MODEL']

        print(f"[bold yellow]Using Local Ollama Model: {config['OPENAI_CHAT_MODEL']}[/bold yellow]")

    else:
        raise ValueError("Unknown OPENAI_API_TYPE, should be azure or openai")

def get_env_config(config):
    return {
        'highway-v0': {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": config["vehicle_count"],
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(5, 32, 9),
            },
            "lanes_count": 4,
            "other_vehicles_type": config["other_vehicle_type"],
            "duration": config["simulation_duration"],
            "vehicles_density": config["vehicles_density"],
            "show_trajectories": True,
            "render_agent": True,
            "scaling": 5,
            'initial_lane_id': None,
            "ego_spacing": 4,
        }
    }

# Helper to display video in notebook
def show_video(result_folder, prefix):
    mp4list = glob.glob(f'{result_folder}/{prefix}*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = open(mp4, 'rb').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

# Run Setup
setup_env_vars(config)
env_config = get_env_config(config)
test_list_seed = [5838, 2421, 7294, 9650, 4176, 6382, 8765, 1348]

if not os.path.exists(config["result_folder"]):
    os.makedirs(config["result_folder"])

print(f"[bold green]Setup Complete.[/bold green] Memory Path: {config['memory_path']}")

Using Local Ollama Model: qwen2.5:14b

Setup Complete. Memory Path: ./memory_db

In [5]:
# Cell 2: Initialize Agents and Memory
# Only run this once to load the DB
REFLECTION = config["reflection_module"]
memory_path = config["memory_path"]
few_shot_num = config["few_shot_num"]

# Initialize Memory
agent_memory = DrivingMemory(db_path=memory_path)

if REFLECTION:
    updated_memory = DrivingMemory(db_path=memory_path + "_updated")
    try:
        updated_memory.combineMemory(agent_memory)
        print("[cyan]Memory combined successfully.[/cyan]")
    except Exception as e:
        print(f"[yellow]Note on memory combination:[/yellow] {e}")

print(f"[bold]Agents initialized.[/bold] Reflection mode: {REFLECTION}")

Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given


==========Loaded  ./memory_db  Memory, Now the database has  0  items.==========
==========Loaded  ./memory_db_updated  Memory, Now the database has  0  items.==========
Merge complete. Now the database has  0  items.


Memory combined successfully.

Agents initialized. Reflection mode: True

In [6]:
# Cell 3: Main Simulation Loop
episode = 0
result_folder = config["result_folder"]
lanes_count = env_config['highway-v0']['lanes_count']

# Logging start
with open(result_folder + "/" + 'log.txt', 'a') as f:
    f.write(f"Batch Start | memory_path {memory_path} | few_shot_num: {few_shot_num} \n")

while episode < config["episodes_num"]:
    print(f"\n[bold magenta]=== Starting Episode {episode} ===[/bold magenta]")

    # 1. Setup Highway Env
    envType = 'highway-v0'
    env = gym.make(envType, render_mode="rgb_array")
    env.configure(env_config[envType])

    result_prefix = f"highway_qwen2.5_14b_ep{episode}"

    # Video Wrapper
    env = RecordVideo(env, result_folder, name_prefix=result_prefix)
    env.unwrapped.set_record_video_wrapper(env)

    seed = random.choice(test_list_seed)
    obs, info = env.reset(seed=seed)

    # 2. Scenario and Agents
    database_path = f"{result_folder}/{result_prefix}.db"
    sce = EnvScenario(env, envType, seed, database_path)
    DA = DriverAgent(sce, verbose=True) # Set verbose=False to reduce notebook clutter
    if REFLECTION:
        RA = ReflectionAgent(verbose=True)

    # 3. Simulation Loop
    response = "Not available"
    action = "Not available"
    docs = []
    collision_frame = -1
    already_decision_steps = 0

    try:
        for i in range(0, config["simulation_duration"]):
            obs = np.array(obs, dtype=float)

            # Retrieve Memory
            fewshot_results = []
            if few_shot_num > 0:
                fewshot_results = agent_memory.retriveMemory(sce, i, few_shot_num)
                print(f"Step {i}: Found {len(fewshot_results)} similar memories.")

            fewshot_messages = [res["human_question"] for res in fewshot_results]
            fewshot_answers = [res["LLM_response"] for res in fewshot_results]

            # Scenario Description
            sce_descrip = sce.describe(i)
            avail_action = sce.availableActionsDescription()

            # Decision
            action, response, human_question, fewshot_answer = DA.few_shot_decision(
                scenario_description=sce_descrip,
                available_actions=avail_action,
                previous_decisions=action,
                fewshot_messages=fewshot_messages,
                driving_intensions="Drive safely and avoid collisons",
                fewshot_answers=fewshot_answers,
            )

            # Store Doc
            docs.append({
                "sce_descrip": sce_descrip,
                "human_question": human_question,
                "response": response,
                "action": action,
                "sce": copy.deepcopy(sce)
            })

            # Step Environment
            obs, reward, done, info, _ = env.step(action)
            already_decision_steps += 1

            # Commit Prompt & Update Video
            sce.promptsCommit(i, None, done, human_question, fewshot_answer, response)
            env.video_recorder.capture_frame()

            if done:
                print(f"[bold red]CRASH / DONE at step {i}[/bold red]")
                collision_frame = i
                break

    except Exception as e:
        print(f"[red]Error during simulation:[/red] {e}")
        import traceback
        traceback.print_exc()

    finally:
        env.close()

        # Log results
        with open(result_folder + "/" + 'log.txt', 'a') as f:
            f.write(f"Episode {episode} | Seed {seed} | Steps: {already_decision_steps}\n")

        # 4. Reflection Logic (Interactive Input)
        if REFLECTION:
            print("[yellow]--- Reflection Phase ---[/yellow]")
            if collision_frame != -1:
                # Crash Case
                print(f"Analyzing crash at frame {collision_frame}")
                for i in range(collision_frame, -1, -1):
                    # Simple heuristic: look for steps where we didn't decelerate (action 4 usually)
                    if docs[i]["action"] != 4:
                        print(f"Found suspicious action at step {i}")
                        corrected_response = RA.reflection(docs[i]["human_question"], docs[i]["response"])
                        print(f"[bold]Suggested Correction:[/bold] {corrected_response}")

                        # Notebook Input
                        choice = input("Add this correction to memory? (Y/N): ").strip().upper()
                        if choice == 'Y':
                            updated_memory.addMemory(
                                docs[i]["sce_descrip"], docs[i]["human_question"],
                                corrected_response, docs[i]["action"],
                                docs[i]["sce"], comments="mistake-correction"
                            )
                            print("[green]Memory updated.[/green]")
                        break
            else:
                # Success Case
                print(f"Episode successful. {len(docs)//5} potential memories to add.")
                choice = input("Add successful memories? (Y/N): ").strip().upper()
                if choice == 'Y':
                    cnt = 0
                    for i in range(0, len(docs)):
                        if i % 5 == 1: # Subsample memories
                            updated_memory.addMemory(
                                docs[i]["sce_descrip"], docs[i]["human_question"],
                                docs[i]["response"], docs[i]["action"],
                                docs[i]["sce"], comments="no-mistake-direct"
                            )
                            cnt +=1
                    print(f"[green]Added {cnt} new memories.[/green]")

    # Show the video of the run
    print(f"Episode {episode} Video:")
    show_video(result_folder, result_prefix)

    episode += 1

=== Starting Episode 0 ===

Use OpenAI API

Cautious: Reflection mode uses OpenAI GPT-4, may cost a lot of money!

Error during simulation: invalid input type

Traceback (most recent call last):
  File "C:\Users\WiCon\AppData\Local\Temp\ipykernel_51592\4127527668.py", line 48, in <module>
    fewshot_results = agent_memory.retriveMemory(sce, i, few_shot_num)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WiCon\Desktop\DiLu\dilu\driver_agent\vectorStore.py", line 43, in retriveMemory
    similarity_results = self.scenario_memory.similarity_search_with_score(
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WiCon\anaconda3\envs\DiLu\Lib\site-packages\langchain\vectorstores\chroma.py", line 432, in similarity_search_with_score
    query_embedding = self._embedding_function.embed_query(query)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WiCon\anaconda3\envs\DiLu\Lib\site-packages\langchain\embeddings\openai.py", line 578, in embed_query
    return self.embed_documents([text])[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

...

MoviePy - Done !
MoviePy - video ready C:\Users\WiCon\Desktop\DiLu\examples\results\highway_qwen2.5_14b_ep0-episode-0.mp4


--- Reflection Phase ---

Episode successful. 0 potential memories to add.

Added 0 new memories.

Episode 0 Video: